In [1]:
import tools
from tools.imports import *
from tools.document_tools import GoogleDocWriteTool
from tools.file_mgmt_tools import MoveFileTool, CreateFolderTool, FolderMovementTool, FileOrganizerTool, ImprovedSearchTool
from tools.miscellaneous_mgmt import GoogleDriveUploadTool, GmailSendPdfTool, GoogleSheetsUpdateTool
from tools.initialize_groq import init_groq
from dotenv import load_dotenv

In [2]:
client, llm = init_groq()

In [3]:
my_tools = []
load_dotenv()

credentials_path = os.getenv('CREDENTIALS_PATH')
my_tools.append(
    
    GoogleDocWriteTool(credentials_path)
)

my_tools.append(
    
    GoogleDriveUploadTool(credentials_path)
)

my_tools.append(
    
    GoogleSheetsUpdateTool(credentials_path)
)

my_tools.append(
    GmailSendPdfTool(credentials_path)
)

my_tools.append(
    MoveFileTool(credentials_path)
)

my_tools.append(
    CreateFolderTool(credentials_path)
)

my_tools.append(
    FolderMovementTool(credentials_path)
)

my_tools.append(
    FileOrganizerTool(credentials_path)
)

my_tools.append(
    ImprovedSearchTool(credentials_path)
)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57750%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocuments+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=dsf3WdIIZcjARLMyk21VCuqtYCJuve&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57758%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=RJKeeA939Ka4l3wkmWjHOc4UGXgWYm&access_type=offline
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A577

In [4]:
from langchain.agents import load_tools
humantool = load_tools(
    ["human"],
    llm=llm,
)[0]
my_tools.append(humantool)

In [5]:
# Chat history tracking
chat_history = []

In [6]:
chat_history

[]

In [9]:
import tools.initialize_groq
import random
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType, AgentExecutor
from google.cloud import texttospeech
from langchain.agents import load_agent


# Define the template
template = """
YOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.
WHEN USER INDICATES THEY WANT TO MOVE SOMETHING INTO GOOGLE DRIVE OR MY DRIVE, YOU PASS IN 'ROOT'!!!!! OKAY!!!!!
RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KNOW YOU HAVE EVERYTHING YOU NEED.
DO NOT ASSUME USER WANTS TO DO ANYTHING AT ALL UNLESS YOU ARE 100% SURE!!!!! UNDERSTAND??????!!!!!! OR ELSE I WILL BECOME ANGRY
BE REALLY CAREFUL WITH FILE AND FOLDER ID'S! ANY WRONG ID'S WILL RESULT IN FAILURE OF OPERATIONS

If user tells you to do something that is not one of these tools/operations,\
      you kindly say that you don't have access to that functionality.

"""



# User credentials
credentials = {
    "name": "Gautham Ramachandran",
    "email": "sriramnallani35@gmail.com",
    "recemail": "gauthamramachandran3@gmail.com",
    "phone": "5715996302"
}


def synthesize_speech(text, output_file):
    print('THE TEXT: ', text)
    idx = text.rfind('@')
    text = text[idx + 1:].strip()
    print('MODIFIED RESPONSE = ', text)
    tts_service_acct_path = os.getenv('SERVICE_ACCOUNT_PATH')
    client = texttospeech.TextToSpeechClient.from_service_account_file(tts_service_acct_path)

    
    input_text = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.MALE
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = client.synthesize_speech(
        input=input_text,
        voice=voice,
        audio_config=audio_config
    )
    with open(output_file, "wb") as out:
        out.write(response.audio_content)
    print(f'Audio content written to "{output_file}"')


agent = None
# Define a function to run the agent and handle the response
def run_agent(input_text):
    global chat_history
    input_text += "ALWAYS GENERATE THE GOOGLE DOC CONTENT YOURSELF! NEVER SPEAK TO HUMAN UNLESS INSTRUCTED. \
            DO ONLY WHAT IS SAID IN INSTRUCTIONS, AND FORMAT YOUR RESPONSES ONLY AS SAID IN INSTRUCTIONS OR ELSE LIFE WILL END AS WE KNOW IT. \
            DO EVERYTHING SAID HERE. NOT EVEN ONE THING SHALL BE LEFT INCOMPLETE.\
            DO NOT CREATE FOLDERS WHEN ORGANIZING FILES.\
            UNLESS USER TELLS YOU TO WRITE SOMETHING IN GOOGLE DOC DO NOT PASS ANYTHING INTO 'input_text' PARAMETER! \
            ALSO, WHEN MOVING A FOLDER INTO ANOTHER FOLDER, ALWAYS USE FOLDER MOVEMENT TOOL\n" + template + "\n"
    thepath = "C:\\DEV\\Webdevfolder\\realestateai\\documents\\renovationcontract.pdf"
    input_text += ("\n\n Here is extra info you will need: \nCredentials:\n" + str(credentials) + "\n"
                   + "Path to file on local system:\n" + thepath + "\nTHE CHAT HISTORY: \n"+str(chat_history))
    print(input_text)
    llm.groq_api_key = random.choice(tools.initialize_groq.api_keys)

    agent = initialize_agent(
        tools=my_tools,
        llm=llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
        verbose=True,
        max_iterations=1000,
        return_intermediate_steps=True
    )
    
    result = agent.invoke({
        "input": input_text,
    })
    chat_history.append({"input": input_text, "response": result})
    mystr = (str(result['intermediate_steps']) + "\n" + str(result['output']))
    
    
    synthesize_speech(client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content":"please sanitize this input so that someone can speak it. START THE SPEAKABLE INPUT WITH '@' symbol: " + mystr
            }
        ],
        model='llama3-70b-8192',
    ).choices[0].message.content, "intermediateoutput.mp3"
    )

    return result

user_input = f""" 
move the folder called 'colab notebooks' into 'fcps google backup'
""" 
result = run_agent(user_input)
print(result['output'])


 
move the folder called 'colab notebooks' into 'fcps google backup'
ALWAYS GENERATE THE GOOGLE DOC CONTENT YOURSELF! NEVER SPEAK TO HUMAN UNLESS INSTRUCTED.             DO ONLY WHAT IS SAID IN INSTRUCTIONS, AND FORMAT YOUR RESPONSES ONLY AS SAID IN INSTRUCTIONS OR ELSE LIFE WILL END AS WE KNOW IT.             DO EVERYTHING SAID HERE. NOT EVEN ONE THING SHALL BE LEFT INCOMPLETE.            DO NOT CREATE FOLDERS WHEN ORGANIZING FILES.            UNLESS USER TELLS YOU TO WRITE SOMETHING IN GOOGLE DOC DO NOT PASS ANYTHING INTO 'input_text' PARAMETER!             ALSO, WHEN MOVING A FOLDER INTO ANOTHER FOLDER, ALWAYS USE FOLDER MOVEMENT TOOL

YOU ARE A VERY ADVANCED DOCUMENT MANAGER WHO USES GOOGLE DRIVE FOR DOCUMENT MANAGEMENT.
WHEN USER INDICATES THEY WANT TO MOVE SOMETHING INTO GOOGLE DRIVE OR MY DRIVE, YOU PASS IN 'ROOT'!!!!! OKAY!!!!!
RESPOND IN A CLEAR CUT MANNER.
DO NOT SAY THINGS LIKE - 'here is the response' and the like. OKAY!!?!??
YOU SHALL NOT INDICATE ANY TOOL USE UNTIL YOU KN